In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Pfade zu den Trainingsdaten
train_data_dir = 'photos_reduced/Karotte_reduced'
validation_data_dir = 'photos_reduced/Zwiebel_reduced'
test_data_dir = 'photos_reduced/Kartoffel_reduced'

# Bildgröße und Batch-Größe
img_width, img_height = 125, 125
batch_size = 32

# Daten vorverarbeiten und erweitern
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Vortrainiertes Modell laden
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Modellarchitektur anpassen
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Nur die oberen Schichten trainieren
for layer in base_model.layers:
    layer.trainable = False

# Modell kompilieren
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modell trainieren
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10)

# Testdaten vorverarbeiten
# Calculate the steps_per_epoch value
steps_per_epoch = train_generator.samples // batch_size

# Train the model with the correct steps_per_epoch value
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=10)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


ValueError: Unexpected value for `steps_per_epoch`. Received value is 0. Please check the docstring for `model.fit()` for supported values.